In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct 23 21:01:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# install huggingface libraries
!pip install pytorch-pretrained-bert pytorch-nlp pytorch_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 60.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.4 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=f6f0077ee3e21e5e68827624d8366629c829bd0ac9795ac8f8716a5d2f059704
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig, BertModel
from pytorch_transformers import AdamW, BertForQuestionAnswering
from tqdm import tqdm, trange
import pandas as pd
import io
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA A100-SXM4-40GB'

In [5]:
!pip install transformers
!pip install torch
!pip install TQDM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.4 MB/s eta 0:00:00


In [6]:
!pip install tokenizers

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import json
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
from transformers import BertTokenizerFast

# 1. Load the data
with open("/content/drive/MyDrive/NLP Final Project/train-v1.1.json", "r") as train_file:
    train_data = json.load(train_file)['data']

with open("/content/drive/MyDrive/NLP Final Project/dev-v1.1.json", "r") as dev_file:
    dev_data = json.load(dev_file)['data']

In [10]:
import json
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
from transformers import BertTokenizerFast

# 1. Load the data
with open("/content/drive/MyDrive/NLP Final Project/train-v1.1.json", "r") as train_file:
    train_data = json.load(train_file)['data']

with open("/content/drive/MyDrive/NLP Final Project/dev-v1.1.json", "r") as dev_file:
    dev_data = json.load(dev_file)['data']

# 2. Initialize the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


# 3. Data Processing function
def process_data(data):
    tokenized_inputs = []
    start_positions = []
    end_positions = []
    for article in data:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answer = qa['answers'][0]

                answer_start_char = answer['answer_start']
                answer_end_char = answer_start_char + len(answer['text'])
                answer_text = answer['text']

                # If the answer can't be found in the context, skip this example.
                actual_answer_start = context.find(answer_text)
                if actual_answer_start == -1:
                    print(f"Couldn't find answer: '{answer_text}' in context. Skipping.")
                    continue

                # Tokenize with overflow handling and stride
                encoded = tokenizer.encode_plus(
                  question, context, max_length=512,
                  return_tensors="pt", truncation="only_second",
                  return_overflowing_tokens=True, stride=128,padding='max_length'
)



                # If no overflowing tokens, handle normally
                if 'overflow_to_sample_mapping' not in encoded:
                    tokenized_inputs.append(encoded)
                    start_positions.append(encoded.char_to_token(answer_start_char))
                    end_positions.append(encoded.char_to_token(answer_end_char))
                else:
                    # Handle cases with overflowing tokens
                    for i in range(len(encoded['input_ids'])):
                        input_ids = encoded['input_ids'][i]
                        tokenized_input = {
                            'input_ids': input_ids,
                            'attention_mask': encoded['attention_mask'][i]
                        }
                        # Adjust start and end positions
                        start_pos = encoded.char_to_token(answer_start_char)
                        end_pos = encoded.char_to_token(answer_end_char)
                        # Check if the answer is in this segment
                        if start_pos is not None and end_pos is not None:
                            tokenized_inputs.append(tokenized_input)
                            start_positions.append(start_pos)
                            end_positions.append(end_pos)

    return tokenized_inputs, start_positions, end_positions






# 4. Process the data
train_tokenized_inputs, train_start_positions, train_end_positions = process_data(train_data)
dev_tokenized_inputs, dev_start_positions, dev_end_positions = process_data(dev_data)


# 5. Define the training parameters
epochs = 12
batch_size = 32

# 6. Filter out any missing data
filtered_data = [(input, start, end) for input, start, end in zip(train_tokenized_inputs, train_start_positions, train_end_positions) if input is not None and start is not None and end is not None]

train_tokenized_inputs = [item[0] for item in filtered_data]
train_start_positions = [item[1] for item in filtered_data]
train_end_positions = [item[2] for item in filtered_data]

# 7. Create DataLoader using `batch_size`
train_loader = DataLoader(list(zip(train_tokenized_inputs, train_start_positions, train_end_positions)), batch_size=batch_size, shuffle=True)

# 8. Initialize the BERT model for Question Answering
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

# 9. Define the device and move the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 10. Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# 11. Define a function to save checkpoints
def save_checkpoint(model, optimizer, epoch, filename):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


# 12. Training loop with TQDM and checkpointing
for epoch in range(epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}", position=0, leave=True)
    for batch in progress_bar:
        inputs, start_positions, end_positions = batch
        inputs = {key: value.to(device) for key, value in inputs.items()}
        start_positions = start_positions.to(device)
        end_positions = end_positions.to(device)

        outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Update the progress bar
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    # Save checkpoint at the end of each epoch
    save_checkpoint(model, optimizer, epoch, f"bert_qa_epoch{epoch+1}.pth")

    # Print average loss for the epoch
    print(f"Epoch {epoch+1} loss: {total_loss/len(train_loader)}")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 192/192 [02:13<00:00,  1.43it/s, training_loss=0.709]


Epoch 1 loss: 2.3351705788324275


Epoch 2: 100%|██████████| 192/192 [02:10<00:00,  1.47it/s, training_loss=0.659]


Epoch 2 loss: 1.870836079120636


Epoch 3: 100%|██████████| 192/192 [02:10<00:00,  1.47it/s, training_loss=0.422]


...

Epoch 10 loss: 0.08081885808496736


Epoch 11: 100%|██████████| 192/192 [02:10<00:00,  1.47it/s, training_loss=0.007]


Epoch 11 loss: 0.06726589810811372


Epoch 12: 100%|██████████| 192/192 [02:10<00:00,  1.47it/s, training_loss=0.029]


Epoch 12 loss: 0.06335095637768973


In [11]:
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(pred_start, pred_end, true_start, true_end):
    f1 = f1_score(true_start, pred_start, average='macro') + f1_score(true_end, pred_end, average='macro')
    exact_match = accuracy_score(true_start, pred_start) * accuracy_score(true_end, pred_end)
    return {"f1": f1/2, "exact_match": exact_match}

dev_loader = DataLoader(list(zip(dev_tokenized_inputs, dev_start_positions, dev_end_positions)), batch_size=batch_size)
model.eval()
all_start_preds, all_end_preds = [], []
with torch.no_grad():
    for batch in dev_loader:
        inputs, start_positions, end_positions = batch
        inputs = {key: value.to(device) for key, value in inputs.items()}
        outputs = model(**inputs)
        start_preds = torch.argmax(outputs.start_logits, dim=1)
        end_preds = torch.argmax(outputs.end_logits, dim=1)
        all_start_preds.extend(start_preds.tolist())
        all_end_preds.extend(end_preds.tolist())

metrics = compute_metrics(all_start_preds, all_end_preds, dev_start_positions, dev_end_positions)
print(metrics)


{'f1': 0.1433867756576912, 'exact_match': 0.1205764644609523}


In [19]:
def chatbot():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    while True:
        model.to('cuda')

        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Tokenize the user's input
        inputs = tokenizer("question: " + user_input, return_tensors="pt")
        inputs = {key: value.to(device) for key, value in inputs.items()}

        # Get the model's answer
        answer = model(**inputs)

        # Extract and print the answer
        answer_start = torch.argmax(answer.start_logits)
        answer_end = torch.argmax(answer.end_logits) + 1
        answer_text = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])
        print("Chatbot:", answer_text)

In [21]:
if __name__ == "__main__":
    print("Chatbot: Hello! You can ask me questions. Type 'exit' to end the conversation.")
    chatbot()

Chatbot: Hello! You can ask me questions. Type 'exit' to end the conversation.
Chatbot: did super
Chatbot: miami was
Chatbot: was created at notre dame
You: exit
Chatbot: Goodbye!
